An input for acceptor site should be a 140-mer string with the AG at positions 69 and 70

An input for donor site should be a 140-mer string with the GT at positions 71 and 72

# Imports

In [ ]:
import pandas as pd
import numpy as np
import pybedtools
from Bio import SeqIO
import hgvs
from hgvs.easy import *

# Functions

In [ ]:
def reverse_sequence(s):
    ''' Converts a sequence into the sequence of the complementary strand'''
    new_sequence = ''
    for base in s:
        if base == 'A':
            new_sequence = new_sequence + 'T'
        elif base == 'T':
            new_sequence = new_sequence + 'A'
        elif base == 'G':
            new_sequence = new_sequence + 'C'
        elif base == 'C':
            new_sequence = new_sequence + 'G'
        else:
            new_sequence = new_sequence + base
    return new_sequence[::-1]

# Define Variables

In [ ]:
gene = 'MYBPC3'
variant = 'NCSS'

length = 140
excel_file = '../variants_scores.xlsx

if gene == 'ABCA4':
    genome = 'GRCh37'
    chromosome = 1
    reverse = True
    # reference fasta can be downloaded from http://ftp.ensembl.org/pub/release-75/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.75.dna.chromosome.1.fa.gz
    reference_fasta = '../Homo_sapiens.GRCh37.dna.chromosome.1.fa'
    if variant == 'NCSS':
        dataset = 'ABCA4_NCSS'
        sheet_name = 'NCSS'
        exon_var = ['c.4538A>C', 'c.6478A>G', 'c.4633A>G', 'c.6385A>G', 'c.3812A>G', 'c.4538A>G']
    elif variant == 'DI':
        dataset = 'ABCA4_DI'
        sheet_name = 'DI'
        exon_var = []
        
elif gene == 'MYBPC3':
    genome = 'GRCh37'
    chromosome = 11
    reverse = True
    # reference fasta can be downloaded from https://www.ncbi.nlm.nih.gov/nuccore/NC_000011.9?report=fasta
    reference_fasta = '../chr11.fa'
    if variant == 'NCSS':
        dataset = 'MYBPC3_NCSS'
        sheet_name = 'NCSS'
        exon_var = ['c.1456T>G', 'c.1789C>T', 'c.2307C>T', 'c.2601C>A', 'c.2601C>G', 'c.405A>G', 'c.2993A>G', 'c.853G>A']


# Read in the data 

In [ ]:
# Read the second column of the excel sheet and store the variants in a list
df = pd.read_excel(excel_file, sheet_name ,index_col=None, usecols = 'B')
variant_list = df['genomic variant'].tolist()

# Store the cDNA variant information
df2 = pd.read_excel(excel_file, sheet_name ,index_col=None, usecols = 'A')
cDNA = df2['cDNA variant'].tolist()

# Also store if it is a donor or acceptor site
df3 = pd.read_excel(excel_file, sheet_name ,index_col=None, usecols = 'D')
ss = df3['affects'].tolist()

# 1) Store the variant information

In [ ]:
info = []

hp = hgvs.parser.Parser()
for i in range(len(variant_list)):
    variant = variant_list[i]
    v = hp.parse_hgvs_variant('Chr' + str(chromosome) + genome + ':' + variant)
    
    # Store the variant information so that it can be accessed separately
    var_info = []
    var_info.append(v.posedit.pos.start.base)
    var_info.append(v.posedit.pos.end.base)
    var_info.append(v.posedit.edit.ref)
    var_info.append(v.posedit.edit.alt)
    
    # add if it affects a donor or acceptor
    var_info.append(ss[i])

    # check if the variant is located upstream or downstream of the splice site
    cdna = cDNA[i]
    if '-' in cdna:
        # check if it is a deletion
        if 'del' in cdna:
            # depending on the length of the deletion (on or multiple bases) a different handling is required
            if '_' in cdna:
                loc = int((cdna.split('-')[1]).split('_')[0])
            else:
                print(cdna)
                loc = int((cdna.split('-')[1][:-3]))
        # if the varant is not a deletion, the cdna string can be split to get the location of the variant
        else:
            loc = int((cdna.split('-')[1]).split('>')[0][:-1])

    # perform the same actions for the downsstream variants but now split the string at + instead of -
    elif '+' in cdna:
        if 'del' in cdna:
            if '_' in cdna:
                loc = int((cdna.split('+')[1]).split('_')[0])
            else:
                loc = int((cdna.split('+')[1][:-3]))
                
        else:
            loc = int((cdna.split('+')[1]).split('>')[0][:-1])            
    else:
        loc = 0
        
    # if the variant is located at the second last position of the exon, a different location is needed (-1)    
    if cdna in exon_var:
        var_info.append(-1)
    else:
        var_info.append(loc)
    
    info.append(var_info)
    

# 2) Create the BED file

In [ ]:
# The BED file defines the sequence range that is written to the fasta file later on
with open ((dataset + '.bed'), 'w') as file:
    for i in range(len(info)):
        # An input for acceptor site should be a 140-mer string with the AG at positions 69 and 70
        if ss[i] == 'acceptor':
            var_loc = info[i][0]
            loc = var_loc - info[i][5] + 1
        
        # An input for donor site should be a 140-mer string with the GT at positions 71 and 72
        else:
            var_loc = info[i][0]
            loc = var_loc + info[i][5]
            
        # If the variant contains a deletion, a longer sequence is needed to end up with a 140nt long sequence
        if 'del' in cDNA[i]:
            # get the length of the deletion
            l = info[i][1] - info[i][0]
            file.write('chr' + str(chromosome) + '\t' + str(loc-(length//2+2)) + '\t' + str(loc+(length//2-1)+l) + '\t\t\t' + '-' + '\n')
            
        else:     
            file.write('chr' + str(chromosome) + '\t' + str(loc-(length//2+1)) + '\t' + str(loc+(length//2-1)) + '\t\t\t' + '-' + '\n')

# 3) Get the sequence for each variant and store it in a fasta file

In [ ]:
a = pybedtools.BedTool((dataset + '.bed'))
a = a.sequence(fi = reference_fasta, fo = (dataset + '.fa.out'))

# 4) Change the mutated base in the fasta sequence

In [ ]:
fasta_sequences = SeqIO.parse(open((dataset + '.fa.out')),'fasta')

# open the new fasta file to save the mutated sequences
with open ((dataset + '_donor.fa.out'), 'w') as file:
    with open ((dataset + '_acceptor.fa.out'), 'w') as file2:
        i = 0
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            
            print(cDNA[i])

            # write the reference sequence to the file
            if reverse == True:
                wt_sequence = reverse_sequence(sequence)[:140]

            else:
                wt_sequence = sequence

            # print([m.start() for m in re.finditer('GT', wt_sequence)])
            if ss[i] == 'acceptor':
                #print(wt_sequence[68:70])
                assert wt_sequence[68:70] == 'AG'
            else:
                #print(cDNA[i], wt_sequence[70:72])
                assert wt_sequence[70:72] in ['GT','GC']

            # get the location of the variant
            if ss[i] == 'acceptor':
                loc = length//2 + info[i][5] - 1
            else:
                loc = length//2 - info[i][5]

            # variants where one base is changed
            if info[i][0] == info[i][1] and info[i][2] != '':
                assert sequence[loc] == info[i][2]
                # change the base at the variant position
                l = list(sequence)
                l[loc] = info[i][3]
                s = ''.join(l)
                # test if the base at the variant position in the sequence is now the same as the mutated base
                assert s[loc] == info[i][3]  

            # filter for variants where one single base is deleted
            elif info[i][0] == info[i][1]:
                s = sequence[:loc+1] + sequence[(loc+2):]

            # handle deletions with more bases
            else:
                size = info[i][1] - info[i][0] + 1
                s = sequence[:loc + 1] + sequence[(loc + size + 1):]

            # write the result to a file
            if reverse == True:
                s = reverse_sequence(s)
                
            if ss[i] == 'acceptor':
                file2.write('>' + cDNA[i] + '\n' + wt_sequence + '\n') 
                file2.write('>' + cDNA[i] + '_var\n' + s + '\n') 
            else:
                file.write('>' + cDNA[i] + '\n' + wt_sequence + '\n') 
                file.write('>' + cDNA[i] + '_var\n' + s + '\n') 

            i += 1